In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm import tqdm
from keras.preprocessing import image

In [2]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_preprocess

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as inception_resnet_v2_preprocess

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa

In [4]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                 "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

In [5]:
width = 299
height = 299

num_aug = 20

In [6]:
TTA = {
    'xception': False,
    'inception_resnet_v2': False,
    'efficientnet': False,
    'cnn': True,
}

In [7]:
# Path to test images
test_dir = '../input/plant-seedlings-classification/'
test_dir_seg = '../input/plant-seedling-segmented/plant-seedling-segmented/'

# Path to saved models
model_dir = '../input/plant-seedling-models/'

In [8]:
def create_datagen(preprocess_fn):
    return ImageDataGenerator(preprocessing_function=preprocess_fn,
                             rotation_range=360,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True
                             )

In [9]:
def create_xception():
    model_main = Xception(weights='imagenet', input_shape=(width, height, 3), include_top=False)
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)

    dropout_1 = tf.keras.layers.Dropout(0.2)(layer)
    model_dense2 = tf.keras.layers.Dense(256, activation = 'selu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.2)(model_dense2)
    model_dense3 = tf.keras.layers.Dense(128, activation='selu')(dropout_2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(model_dense3)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [10]:
def create_inception_resnet_v2():
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=(width, height, 3), include_top=False, weights='imagenet')(model_input)
            
    model_dense1 = GlobalAveragePooling2D(name="avg_pool")(model_main)
    model_dense1 = BatchNormalization()(model_dense1)

    model_dense2 = tf.keras.layers.Dense(256, activation =  tfa.activations.mish, activity_regularizer=tf.keras.regularizers.l2(1e-5))(model_dense1)
    model_dense2 = BatchNormalization()(model_dense2)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [11]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten

def create_efficientnet():
    
    model_main = tf.keras.applications.efficientnet.EfficientNetB3(input_shape=(width, height, 3), include_top=False, weights='imagenet')
    
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    layer = Dropout(0.2, name="top_dropout")(layer)

    model_dense2 = tf.keras.layers.Dense(256, activation = 'selu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(layer)
    model_dense2 = BatchNormalization()(model_dense2)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [12]:
def tta(model, datagen, num_aug, test_path, submission_name, width=299, height=299):
    
    submission = pd.DataFrame()

    for file in tqdm(os.listdir(test_path)):
        img = image.load_img(os.path.join(test_path, file), target_size=(width, height))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        pred = np.zeros([12,])
        for i, im in enumerate(datagen.flow(x)):
            pred += model.predict(im)[0]
            if i >= num_aug:
                break
        submission = submission.append({'file': file, 'species': species_list[np.where(pred==np.max(pred))[0][0]]}, ignore_index=True)

    submission.to_csv(submission_name, index=False)

In [13]:
if TTA['xception']:
    
    datagen = create_datagen(xception_preprocess)
    xception = create_xception()
    xception.load_weights(f'{model_dir}xception.h5')

    tta(xception, datagen, num_aug, f'{test_dir_seg}seg_test/', f'xception_seg_tta_{num_aug}_submission.csv')

In [14]:
if TTA['inception_resnet_v2']:

    datagen = create_datagen(inception_resnet_v2_preprocess)
    
    inception_resnet_v2 = create_inception_resnet_v2()
    inception_resnet_v2.load_weights(f'{model_dir}inception_resnet_v2.h5')

    tta(inception_resnet_v2, datagen, num_aug, f'{test_dir}test/', f'inception_resnet-v2_tta_{num_aug}_submission.csv')

In [15]:
if TTA['efficientnet']:

    datagen = create_datagen(tf.keras.applications.efficientnet.preprocess_input)
    
    efficientnet = create_efficientnet()
    efficientnet.load_weights(f'{model_dir}efficientnetB3_balanced.h5')

    tta(efficientnet, datagen, num_aug, f'{test_dir}test/', f'efficientnetB3_tta_{num_aug}_submission.csv')

In [16]:
def tta_cnn(model, datagen, num_aug, test_path, submission_name, width=128, height=128):
    
    submission = pd.DataFrame()

    for file in tqdm(os.listdir(test_path)):
        image = cv2.imread(os.path.join(test_path, file), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (128,128))
        image = np.expand_dims(image, axis=0)
        pred = np.zeros([12,])
        for i, im in enumerate(datagen.flow(image)):
            pred += model.predict(im)[0]
            if i >= num_aug:
                break
        submission = submission.append({'file': file, 'species': species_list[np.where(pred==np.max(pred))[0][0]]}, ignore_index=True)

    submission.to_csv(submission_name, index=False)

In [17]:
if TTA['cnn']:
    
    from keras.models import load_model
    datagen = ImageDataGenerator(
                             rotation_range=360,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True
                             )
    cnn = load_model(f'{model_dir}cnn.h5')
    

    tta_cnn(cnn, datagen, num_aug, f'{test_dir}test/', f'cnn_tta_{num_aug}_submission.csv', 128, 128)

2022-04-18 15:10:34.087639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 15:10:34.202909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 15:10:34.203698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 15:10:34.204801: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil